In [ ]:
!pip install kaggle
!pip install --upgrade kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c uw-cs480-fall20

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.10)


Saving kaggle.json to kaggle (5).json
uw-cs480-fall20.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls
import zipfile
zip_ref = zipfile.ZipFile('uw-cs480-fall20.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()
import pandas as pd
samples=pd.read_csv('/content/files/sample_submission.csv',dtype=str)
test_data=pd.read_csv('/content/files/test.csv')
train_data=pd.read_csv('/content/files/train.csv')
image_dir = '/content/files/suffled-images'
train_data=train_data.reset_index(drop=True)
test_data=test_data.reset_index(drop=True)


 drive		   'kaggle (4).json'		     MLP_outpout.csv
 files		   'kaggle (5).json'		     sample_data
'kaggle (1).json'   kaggle.json			     uw-cs480-fall20.zip
'kaggle (2).json'   LogisticRegression_outpout.csv
'kaggle (3).json'   MLP.h5


In [ ]:
test_data=pd.read_csv('/content/files/test.csv',dtype=str)
train_data=pd.read_csv('/content/files/train.csv',dtype=str)
image_dir = '/content/files/suffled-images'
train=train_data.reset_index(drop=True)
test=test_data.reset_index(drop=True)
train_folder = '/content/files/'
test_folder = '/content/files/'
samples=pd.read_csv('/content/files/sample_submission.csv',dtype=str)




In [ ]:
def Addjpg(fn):
    return fn+".jpg"
# train["id"]=train["id"].apply(append_ext)
# test["id"]=test["id"].apply(append_ext)

In [ ]:
train.head()

,id,category,gender,baseColour,season,usage,noisyTextDescription
0,36274,Scarves,Women,Grey,Summer,Casual,Femella Women Ankle-Length Grey AQ-S800WD-1EVD...
1,15129,Flip Flops,Unisex,Green,Summer,Casual,Converse Unisex Casual Skirts Slipper
2,58976,Topwear,Women,Red,Summer,Ethnic,Velia Women Acetone Kurta
3,32922,Sandal,Men,Brown,Summer,Casual,Enroute Men Leather Brown Sandals
4,29561,Topwear,Women,Pink,Fall,Ethnic,Aneri Exclusive Anu Pink Inspirartion


In [ ]:
train['category'].unique().shape

(27,)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

MLP model

In [ ]:
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences

sentences = train['noisyTextDescription'].values
labels = train['category'].values

sentences_test = test['noisyTextDescription'].values


encoder = LabelBinarizer()

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences)

tokenizer.fit_on_texts(sentences_test)

sentences = tokenizer.texts_to_sequences(sentences)
sentences_test = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

maxlen = 30


sentences = pad_sequences(sentences, padding='post', maxlen=maxlen)
sentences_test = pad_sequences(sentences_test, padding='post', maxlen=maxlen)

labels = encoder.fit_transform(labels) 


print(sentences[2])


[3852    2   25    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [ ]:
import keras
from keras.models import Sequential
from keras import layers



embedding_dim = 150
n_classes = 27




model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(n_classes, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy'])
model.summary()

history = model.fit(sentences, labels,
                    epochs=5,
                    verbose=1,
                    validation_split=0.15,
                    batch_size=30)
loss, accuracy = model.evaluate(sentences, labels, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 150)           1314000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 4500)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               2304512   
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_8 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_9 (Dense)              (None, 27)               

In [ ]:
model.save('MLP_89.h5')

In [ ]:
# prepare submitfile

import numpy as np


model = keras.models.load_model('MLP.h5')

li = np.argmax(model.predict(sentences_test), axis=-1)
cls = np.zeros((li.shape[0],27))
cls[np.arange(li.size),li] = 1
prediction = encoder.inverse_transform(cls)
prediction.shape

In [ ]:
print(model.predict(sentences))